In [ ]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
cd MyDrive/MyDrive/Capstone/code_data

# Libraries

In [2]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import matplotlib as plt

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from datetime import datetime
from datetime import date
from datetime import timedelta

from nltk import SnowballStemmer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
import emoji


[nltk_data] Downloading package punkt to /Users/june/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/june/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 400)
# pd.set_option('display.max_rowwidth', 100)
pd.set_option('display.max_colwidth', 200)

# Functions

In [35]:
lmt = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
freqdist = nltk.FreqDist()
stop_words = set(stopwords.words('english'))
tweet_tokenizer = TweetTokenizer()

""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not'), 
                         (r'i\'d', 'i would'), (r'I\'d', 'I would'), (r'he\'d', 'he would'), (r'she\'d', 'she would'), (r'they\'d', 'they would'), (r'we\'d', 'we would')]
def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    return capitalratio 

def getTokenization(sent):
    tweet_tokens = []
    sent = sent.lower()
    sent = replaceContraction(sent)

    sent = re.sub(r"http\S+", "*", sent) # http link -> '*'
    # sent = re.sub(r"@\S+", "@", sent)   # mention -> '@'
    sent = re.sub(r"@[^\s]+", "@", sent)   # mention -> '@'
    sent = re.sub(r"(#)(\S+)", r'\1 \2', sent) 

    sent = re.sub(r'([^\s\w@#\*]|_)+', '', sent) # Erasing Special Characters
    # sent = re.sub('@[^\s]+','atUser',sent)
    # sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',sent)
    # sent = re.sub(r'#([^\s]+)', r'\1', sent)


    # sent = re.sub('', '', sent.lower())
    # sent = [tweet_tokenizer.tokenize(sent)]
    sent = tweet_tokenizer.tokenize(sent)
    sent = [stemmer.stem(token) for token in sent]
    sent = [lmt.lemmatize(token) for token in sent]

    temp = [token for token in sent if not token in stop_words]
    url=0
    for token in temp:
        if token.startswith('*'):
            url+=1
    # tweet_tokens.append([temp])
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    # df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    return temp, url

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0
    if len(entities_dicts) == 1:
        return 1
    if len(entities_dicts) == 2:
        return 2

    # urls = []
    # urls_expanded = []

    # key = 'url'
    # key2 = 'expanded_url'
    # # print(len(entities_dict))
    # for i in entities_dicts:
    #     urls.append(i[key])
    #     urls_expanded.append(i[key2])
    # return 1, urls, urls_expanded

def getposcount(tokens):
    postag = []
    poscount = {}
    poscount['Noun']=0
    poscount['Verb']=0
    poscount['Adjective'] = 0
    poscount['Pronoun']=0
    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['Adverb']=0
    poscount['Numeral']=0
    poscount['Conjunction_inj']=0
    poscount['Particle']=0
    poscount['Determiner']=0
    poscount['Modal']=0
    poscount['Whs']=0

    Nouns = {'NN','NNS','NNP','NNPS'}
    Adverbs = {'RB','RBR','RBS'}
    Whs = {'WDT','WP','WRB'} # Composition of wh-determiner(that,what), wh-pronoun(who), wh-adverb(how)
    Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
    first_person_pronouns=['i','I','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours', 'ya']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']
    test_auxiliary=['be','will','have','am','is','was','were','can','could','dare','did','may','might','must','ought','shall','should','would']
    test_tentat=['maybe','perhaps','possibly','probably','guess']
    test_certain=['always','never', "can't", 'cannot']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
        
    for word in tokens:
        w_lower=word.lower()
        if w_lower in test_auxiliary:
            poscount['test_auxiliary']+=1
        elif w_lower in test_tentat:
            poscount['test_tentat']+=1
        elif w_lower in test_certain:
            poscount['test_certain']+=1

    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1] in Nouns:
            poscount['Noun'] += 1
        elif g1[1] in Verbs:
            poscount['Verb']+= 1
        elif g1[1]=='ADJ'or g1[1]=='JJ':
            poscount['Adjective']+=1
        elif g1[1]=='PRP' or g1[1]=='PRON' or g1[1]=='PRP$':
            poscount['Pronoun']+=1
        elif g1[1] in Adverbs or g1[1]=='ADV':
            poscount['Adverb']+=1
        elif g1[1]=='CD':
            poscount['Numeral']+=1
        elif g1[1]=='CC' or g1[1]=='IN':
            poscount['Conjunction_inj']+=1
        elif g1[1]=='RP':
            poscount['Particle']+=1
        elif g1[1]=='MD':
            poscount['Modal']+=1
        elif g1[1]=='DT':
            poscount['Determiner']+=1
        elif g1[1] in Whs:
            poscount['Whs']+=1
    return poscount

def fetchRawText(path, events, tweetType):
    jsons = []
    for i, event in enumerate(events):
        jsons.append(glob('%s/%s/**/%s/*.json' % (path, event,tweetType)))
    for i,d in enumerate(jsons): print("%s's length is %d" %(events[i], len(d)))

    targets = []
    features = []
    for index, dataset in enumerate(jsons):
        targetEvent = []
        dataEvent = []
        count = 0  # help var
        for jsonFile in dataset:
            count += 1
            if jsonFile.find("non-rumours") == -1:
                targetEvent.append(1)
            else:
                targetEvent.append(0)

            with open(jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip():  # skip empty lines
                        continue
                    try:
                        json_data = json.loads(l)
                    except:
                        print (l,"\n\n")
                        break
                    dataEvent.append(json_data)
        print(index, events[index], len(targetEvent), len(dataEvent))
        targets.append(targetEvent)
        features.append(dataEvent)

    # print("\nNumber of Events:", len(targets))
    # print("Number of tweets in the first event:", len(targets[0]))

    # targets은 targetEvent들을 리스트에 담은 것
    target_list = []
    for event in targets:
        for elem in event:
            target_list.append(elem)
    target = pd.DataFrame(target_list, columns=["target"])

    extracted_features = []

    extracted = []

    for obj_list in features:
        extracted_event = []
        for obj in obj_list:
            output_f = dict()
            output_f['text'] = obj['text']
            urls_dicts = obj['entities']['urls']
            # output_f['URLcount'] = extract_urls(urls_dicts)

            # NEW Features from fetchRawText_all
            output_f['emoji_count'] = emoji.emoji_count(obj['text'])
            urls_dicts = obj['entities']['urls']
            output_f['URLcount'] = len(urls_dicts)
            if "media" in obj['entities']:
                output_f['has_media'] = len(obj['entities']['media'])
                # output_f['media_type'] = obj['entities']['media'][0]['type']
            else:
                output_f['has_media'] = 0
            temp = re.sub(r"http\S+", "HTTPURL", obj['text'])
            verification = 0
            verification += len(re.findall(r'is(that|this|it) true', obj['text']))
            verification += len(re.findall(r'wh[a]*t[?!|!?][?!|!?]*', obj['text']))
            verification += len(re.findall(r'(rumour|rumor|debunk)', obj['text']))
            verification += len(re.findall(r'(real?|really?|uncomfirmed)', obj['text']))
            verification += len(re.findall(r'(that|this|it) is not true', obj['text']))
            verification += len(re.findall(r'(that|this|it) is false', obj['text']))
            verification += len(re.findall(r'(h[m]*)', obj['text']))
            output_f['Skepticism'] = verification    
            mention = 0
            for token in temp:
                if token.startswith('@'):
                    mention+=1 
            output_f['MentionCount'] = mention      

            output_f['text_token'], output_f['URLcount'] = getTokenization(obj['text'])
            '''POS Tagging and text cleansing for POS'''
            # temp = output_f['text']
            # temp=  emoji.demojize(temp)
            # temp = re.sub(r"(#)(\S+)", r'\1 \2', temp)
            # temp = re.sub(r"http\S+", "", temp)
            # temp = replaceContraction(temp.lower())
            # temp = temp.split()
            # pos_dict=getposcount(temp)

            '''POS Tagging'''
            temp = output_f['text']
            temp = replaceContraction(temp.lower())
            temp = re.sub(r"(#)(\S+)", '', temp)
            temp = re.sub(r"(@)(\S+)", '', temp)
            temp = re.sub(r"http\S+", "", temp)
            temp = re.sub(r'([^\s\w#\*]|_)+', '', temp) # Erasing Special Characters
            temp = temp.split()
            pos_dict=getposcount_all(temp)
            output_f.update(pos_dict)

            output_f['char_count'] = len(output_f['text'])
            output_f['word_count'] = len(output_f['text'].split())

            output_f['HashTag'] = len(obj['entities']['hashtags'])

            output_f['has_question'] = "?" in output_f["text"]
            output_f['has_exclaim'] = "!" in output_f["text"]
            output_f['has_period'] = "." in output_f["text"]

            output_f['capital_ratio']=(capitalratio(obj['text']))
            output_f['retweet_count'] = obj['retweet_count']
            # output_f['isRT'] = obj['retweeted']

            output_f['tweet_count'] = obj['user']['statuses_count']
            output_f['listed_count'] = obj['user']['listed_count']
            output_f['friends_count'] = obj['user']['friends_count']
            output_f['follower_count'] = obj['user']['followers_count']

            if (output_f['friends_count'] <= 0):
                output_f['followers/friend'] = obj['user']['followers_count']
            else:
                output_f['followers/friend'] = obj['user']['followers_count']/obj['user']['friends_count']

            output_f['favourites_count'] = obj['user']['favourites_count']

            acc_created = datetime.strptime(obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
            tweet_created = datetime.strptime(obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
            age = (tweet_created - acc_created)
            output_f['account_age_days'] = age.days
            
            output_f['capital_ratio']=(capitalratio(obj['text']))
            output_f['verified'] = obj['user']['verified']

            extracted_event.append(output_f)
        extracted_features.append(extracted_event)

    extracted_df = []
    for i, data in enumerate(extracted_features):
        temp = pd.DataFrame(data)
        temp["Event"] = events[i]
        extracted_df.append(pd.DataFrame(temp))

    final = pd.concat(extracted_df, ignore_index=True)
    final = pd.concat([final, target], axis=1)
    return final
    

In [8]:
def getposcount_all(tokens):
    postag = []
    poscount = {}

    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['test_auxiliary']=0
    poscount['test_tentat']=0
    poscount['test_certain']=0
    poscount['Numeral']=0


    first_person_pronouns=['i','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours', 'ya']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']
    test_auxiliary=['be','will','have','am','is','was','were','can','could','dare','did','may','might','must','ought','shall','should','would']
    test_tentat=['maybe','perhaps','possibly','probably','guess']
    test_certain=['always','never']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
        
    for word in tokens:
        w_lower=word.lower()
        if w_lower in test_auxiliary:
            poscount['test_auxiliary']+=1
        elif w_lower in test_tentat:
            poscount['test_tentat']+=1
        elif w_lower in test_certain:
            poscount['test_certain']+=1

    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1]=='CD':
            poscount['Numeral']+=1
    return poscount

# PHEME

In [40]:
path = "../pheme-rnr-dataset"
events = ['charliehebdo', 'ferguson',
          'germanwings-crash', 'ottawashooting', 'sydneysiege']
tweetType = 'source-tweet'
jsons = []
final = fetchRawText(path, events, tweetType)
target = final.target
final.verified = final.verified.replace({True: 1, False: 0}) 
final.has_question = final.has_question.replace({True: 1, False: 0}) 
final.has_exclaim = final.has_exclaim.replace({True: 1, False: 0}) 
final.has_period = final.has_period.replace({True: 1, False: 0}) 
# final.isRT = final.isRT.replace({True: 1, False: 0}) 

final = final.replace(-np.inf, 0)

charliehebdo's length is 2079
ferguson's length is 1143
germanwings-crash's length is 469
ottawashooting's length is 890
sydneysiege's length is 1221
0 charliehebdo 2079 2079
1 ferguson 1143 1143
2 germanwings-crash 469 469
3 ottawashooting 890 890
4 sydneysiege 1221 1221


In [41]:
final.drop(['text_token','text','Event','target'], axis=1, inplace=True)
final.head()

,emoji_count,URLcount,has_media,Skepticism,MentionCount,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,test_auxiliary,test_tentat,test_certain,Numeral,char_count,word_count,HashTag,has_question,has_exclaim,has_period,capital_ratio,retweet_count,tweet_count,listed_count,friends_count,follower_count,followers/friend,favourites_count,account_age_days,verified
0,0,1,0,4,0,0,0,0,0,0,0,0,88,12,0,0,0,1,0.159091,177,63575,7177,614,193798,315.631922,1.886491,2126,1
1,0,0,0,1,0,0,0,0,0,0,0,0,53,6,1,0,0,1,0.037736,134,1076,140,375,4709,12.557333,-10.000000,1050,0
2,0,0,0,3,0,0,0,0,1,0,0,0,136,18,2,0,0,1,0.073529,148,7182,758,592,20401,34.461149,2.173186,2030,0
3,0,2,0,8,0,0,0,0,2,0,0,0,138,16,1,0,0,1,0.101449,684,54427,102287,1038,15405096,14841.132948,2.952792,2891,1
4,0,2,1,3,0,0,0,0,0,0,0,0,117,13,1,0,0,1,0.145299,113,104998,13583,460,842236,1830.947826,1.414973,1975,1


In [42]:
final.to_csv('./data/_PHEME_sparse.csv', index = False)

# PHEME (Extension)

In [36]:
path = "../PHEME/all-rnr-annotated-threads"
events = ['ebola-essien', 'prince-toronto', 'putinmissing']
tweetType = 'source-tweets'
jsons = []
final_ext = fetchRawText(path,events,tweetType)
ext_target = final_ext.target
final_ext.verified = final_ext.verified.replace({True: 1, False: 0}) 
final_ext.has_question = final_ext.has_question.replace({True: 1, False: 0}) 
final_ext.has_exclaim = final_ext.has_exclaim.replace({True: 1, False: 0}) 
final_ext.has_period = final_ext.has_period.replace({True: 1, False: 0}) 
# final_ext.isRT = final_ext.isRT.replace({True: 1, False: 0}) 
final_ext = final_ext.replace(-np.inf, 0)

ebola-essien's length is 14
prince-toronto's length is 233
putinmissing's length is 238
0 ebola-essien 14 14
1 prince-toronto 233 233
2 putinmissing 238 238


In [37]:
print(final_ext.columns)
final_ext.drop(['text_token','text','Event','target'], axis=1, inplace=True)
final_ext.head()

Index(['text', 'emoji_count', 'URLcount', 'has_media', 'Skepticism',
       'MentionCount', 'text_token', 'FirstPersonPronoun',
       'SecondPersonPronoun', 'ThirdPersonPronoun', 'test_auxiliary',
       'test_tentat', 'test_certain', 'Numeral', 'char_count', 'word_count',
       'HashTag', 'has_question', 'has_exclaim', 'has_period', 'capital_ratio',
       'retweet_count', 'tweet_count', 'listed_count', 'friends_count',
       'follower_count', 'followers/friend', 'favourites_count',
       'account_age_days', 'verified', 'Event', 'target'],
      dtype='object')


,emoji_count,URLcount,has_media,Skepticism,MentionCount,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,test_auxiliary,test_tentat,test_certain,Numeral,char_count,word_count,HashTag,has_question,has_exclaim,has_period,capital_ratio,retweet_count,tweet_count,listed_count,friends_count,follower_count,followers/friend,favourites_count,account_age_days,verified
0,0,1,0,5,0,0,0,0,0,0,0,0,69,8,0,0,0,1,0.101449,117,40676,148,652,21833,33.486196,3.984527,1570,0
1,0,0,0,4,0,2,0,0,4,0,0,0,148,25,1,0,0,1,0.027027,10402,509,1623,176,488496,2775.545455,0.845098,579,1
2,0,0,0,4,0,0,0,1,0,0,0,0,119,20,0,0,0,1,0.025210,126,83232,2165,144,232347,1613.520833,2.276462,2042,0
3,0,2,1,9,0,0,0,0,0,0,0,0,130,16,0,0,0,1,0.107692,192,15448,608,716,73548,102.720670,2.004321,468,1
4,0,1,0,6,1,0,0,0,1,0,0,0,120,15,0,0,0,1,0.066667,196,83232,2165,144,232347,1613.520833,2.276462,2039,0


In [39]:
final_ext.to_csv('./data/_PHEMEext_sparse.csv', index = False)

# Feature Importance

In [30]:
def f_imp(X, y):
    forest = ExtraTreesClassifier(n_estimators=250,
                                random_state=3)

    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d: %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

    # Plot the impurity-based feature importances of the forest
    # plt.figure()
    # plt.title("Feature importances")
    # plt.bar(range(X.shape[1]), importances[indices],
    #         color="r", yerr=std[indices], align="center")
    # plt.xticks(range(X.shape[1]), indices)
    # plt.xlim([-1, X.shape[1]])
    # plt.show()

In [31]:
f_imp(final_ext, pheme_y)

NameError: name 'pheme_y' is not defined